# Exploration

In [11]:
import requests

BASE_URL = "https://portail-api-data.montpellier3m.fr"

def get_ecocounter_timeseries(ecocounter_id, from_date, to_date):
    url = f"{BASE_URL}/ecocounter_timeseries/{ecocounter_id}/attrs/intensity"
    params = {
        "fromDate": from_date,  # ex: "2024-01-01T00:00:00"
        "toDate": to_date       # ex: "2024-01-07T23:59:59"
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()

def list_ecocounters():
    url = f"{BASE_URL}/ecocounter"
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

counters = list_ecocounters()
print(f"Nombre d'écocompteurs : {len(counters)}")

ecocounter_id = counters[19]["id"]  # à remplacer par un ID réel issu de l'appel précédent
print(ecocounter_id)
data = get_ecocounter_timeseries(
    ecocounter_id,
    "2024-01-01T00:00:00",
    "2024-01-07T23:59:59"
)

print(data)

Nombre d'écocompteurs : 20
urn:ngsi-ld:EcoCounter:X2H21070346
{'attrName': 'intensity', 'entityId': 'urn:ngsi-ld:EcoCounter:X2H21070346', 'entityType': 'EcoCounter', 'index': ['2024-01-01T00:00:00.000+00:00', '2024-01-01T01:00:00.000+00:00', '2024-01-01T02:00:00.000+00:00', '2024-01-01T03:00:00.000+00:00', '2024-01-01T04:00:00.000+00:00', '2024-01-01T05:00:00.000+00:00', '2024-01-01T06:00:00.000+00:00', '2024-01-01T07:00:00.000+00:00', '2024-01-01T08:00:00.000+00:00', '2024-01-01T09:00:00.000+00:00', '2024-01-01T10:00:00.000+00:00', '2024-01-01T11:00:00.000+00:00', '2024-01-01T12:00:00.000+00:00', '2024-01-01T13:00:00.000+00:00', '2024-01-01T14:00:00.000+00:00', '2024-01-01T15:00:00.000+00:00', '2024-01-01T16:00:00.000+00:00', '2024-01-01T17:00:00.000+00:00', '2024-01-01T18:00:00.000+00:00', '2024-01-01T19:00:00.000+00:00', '2024-01-01T20:00:00.000+00:00', '2024-01-01T21:00:00.000+00:00', '2024-01-01T22:00:00.000+00:00', '2024-01-01T23:00:00.000+00:00', '2024-01-02T00:00:00.000+00:00',

In [4]:
import requests
import pandas as pd

# Liste des compteurs
counters = [
    "urn:ngsi-ld:EcoCounter:X2H25023006",
    "urn:ngsi-ld:EcoCounter:ZLT25011699",
    "urn:ngsi-ld:EcoCounter:COM24010120",
    "urn:ngsi-ld:EcoCounter:COM23120117",
    "urn:ngsi-ld:EcoCounter:X2H24042101",
    "urn:ngsi-ld:EcoCounter:COM23120110",
    "urn:ngsi-ld:EcoCounter:COM23120111",
    "urn:ngsi-ld:EcoCounter:COM23120114",
    "urn:ngsi-ld:EcoCounter:COM23120113",
    "urn:ngsi-ld:EcoCounter:COM23120112",
    "urn:ngsi-ld:EcoCounter:COM24010121",
    "urn:ngsi-ld:EcoCounter:COM24010119",
    "urn:ngsi-ld:EcoCounter:XTH24072390"
]

# Dates pour l'API (format ISO)
from_date = "2025-03-10T00:00:00"
to_date   = "2025-11-28T23:59:59"

all_data = []

for counter_id in counters:
    print(f" Récupération pour : {counter_id}")
    
    # 1️ Historique des séries temporelles
    url_series = f"https://portail-api-data.montpellier3m.fr/ecocounter_timeseries/{counter_id}/attrs/intensity"
    params = {"fromDate": from_date, "toDate": to_date}
    response = requests.get(url_series, params=params)
    
    if response.status_code == 200:
        data_series = response.json()
        df_series = pd.DataFrame({
            "datetime": data_series["index"],
            "intensity": data_series["values"]
        })
        df_series["counter_id"] = counter_id
    else:
        print(f"Erreur séries temporelles pour {counter_id} : {response.status_code}")
        continue

    # 2️ Description du compteur
    url_desc = f"https://portail-api-data.montpellier3m.fr/ecocounter/{counter_id}"
    response_desc = requests.get(url_desc)
    
    if response_desc.status_code == 200:
        data_desc = response_desc.json()
        lat, lon = data_desc["location"]["value"]["coordinates"][0], data_desc["location"]["value"]["coordinates"][1]
        laneId = data_desc.get("laneId", {}).get("value", None)
        vehicleType = data_desc.get("vehicleType", {}).get("value", None)
        
        # Ajouter les colonnes statiques
        df_series["lat"] = lat
        df_series["lon"] = lon
        df_series["laneId"] = laneId
        df_series["vehicleType"] = vehicleType
    else:
        print(f"Erreur description pour {counter_id} : {response_desc.status_code}")
        df_series["lat"] = None
        df_series["lon"] = None
        df_series["laneId"] = None
        df_series["vehicleType"] = None

    all_data.append(df_series)
    print(f"{len(df_series)} lignes récupérées")

# 3️ Fusion finale
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    df_all["datetime"] = pd.to_datetime(df_all["datetime"])
    
    # 4️ Sauvegarde CSV
    df_all.to_csv("../data/ecocounters_full_complet.csv", index=False)
    print("\n CSV complet enregistré : ../data/ecocounters_full_complet.csv")
else:
    print("Aucune donnée récupérée")

 Récupération pour : urn:ngsi-ld:EcoCounter:X2H25023006
6308 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:ZLT25011699
2040 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM24010120
4205 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120117
3993 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:X2H24042101
6311 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120110
4018 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120111
4042 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120114
4090 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120113
4162 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM23120112
4258 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM24010121
369 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:COM24010119
218 lignes récupérées
 Récupération pour : urn:ngsi-ld:EcoCounter:XTH2407239